In [129]:
import pandas as pd
import shutil
import os
import csv
import re
    

def get_heading(df: pd.DataFrame, cols=None):
    """ Accepts a pandas DataFrame and list of columns
        and creates a new 'heading' column.

        :df:
            pandas DataFrame
        :cols:
            list of columns used to make the 'heading'

        :returns: df with 'heading' column added
    """

    # change missing entries to empty strings
    df[cols] = df[cols].fillna('')

    df['heading'] = df[cols].agg('-'.join, axis=1)

    return df


def write_html(df: pd.DataFrame):
    """ Accepts a pandas DataFrame and writes an html file """
    
    # open the html template
    with open('templates/template.html', 'r') as html_file:
        HTML = html_file.read()
        
    # create column to store individual html markup
    df['html'] = HTML


def make_discussions(path: str, program_name: str) -> int:
    """ Accepts a Microsoft Excel file path, builds pandas DataFrame with the 'heading' column,
        and creates the corresponding XML/HTML files for the D2L Discussions.
        
        Stores all HTML files in temp_folder until ready to zip.
    """
    
    try:
        os.mkdir('temp_folder')
    except FileExistsError:
        pass
    

    df = pd.read_excel(path)
    
    # string extra spaces from the columns
    df.columns = list([col.strip() for col in list(df.columns)])
    
    # rename the columns to match the corresponding fields in the html template
    with open('templates/html_fields.csv', 'r') as csvfile:
        fields = dict(list(csv.DictReader(csvfile))[0])
        
    # change the name of the really long-named columns - these have been really problematic
    for col in df.columns:
        if len(col) > 32:
            if re.search('(combined)', col):
                df.rename(columns={col: 'combined_program'}, inplace=True)
            if re.search('(advanced credit)', col):
                df.rename(columns={col: 'advanced_credit'}, inplace=True)
        
    for key in fields.keys():
        df.rename(columns={key: fields[key]}, inplace=True)
        
    # use only the columns that we need
    df = df[fields.values()]
    
    # fill in missing values with empty string
    df.fillna('', inplace=True)
        
    # put names in proper case
    df['first_name'] = df['first_name'].str.title()
    df['last_name'] = df['last_name'].str.title()

    # convert student ID to string
    df['student_number'] = df['student_number'].apply(str)
    
    # copy the index column to an ID column
    df['ID'] = df.index
    df['ID'] = df['ID'].apply(str)

    # create a new column of discussion topics for all students
    with open('templates/DiscussionTemplate.txt', 'r') as single_topic:
        text = single_topic.read()
    df['discussion_topic'] = text
    
    # create a filename column, replacing spaces with empty strings
    df['html_filename'] = df['first_name'].str.replace(' ', '') + df['last_name'].str.replace(' ', '') + df['student_number'] + '.html'
    
    # replace the FILENAME in the discussion with the 'html_filename' values
    df['discussion_topic'] = df.apply(lambda x: x['discussion_topic'].replace('FILENAME', x['html_filename']), axis=1)

    # replace the fields in the discussion topic column with the corresponding student data
    for field in fields.values():
        df['discussion_topic'] = df.apply(lambda x: x['discussion_topic'].replace(field, x[field]), axis=1)
        
    # replace the ID tag in the XML markup
    df['discussion_topic'] = df.apply(lambda x: x['discussion_topic'].replace('IDnumberHERE', x['ID']), axis=1)
    
    return df


def make_forum(df: pd.DataFrame, year_of_application: int):
    """ Accepts a pandas DataFrame of student discussions and creates the D2L 
        discussion forum object by filling in discussion_d2l_TemplateFile.txt
    """
    
    with open('templates/discussion_d2l_TemplateFile.txt', 'r') as all_students:
        forum = all_students.read()
        
    all_topics = ''.join(df['discussion_topic'].to_list())
    forum = forum.replace("InsertTopicHere", all_topics)
    forum = forum.replace("TOPIC_TITLE", str(year_of_application) + " Applications")
    
    with open('temp_folder/d2l_disc.xml', 'w') as file:
        file.write(forum)
        
    return 0

In [130]:
df = make_discussions('PHAS Fall 2022 admission.xlsx', 'PHAS')

In [131]:
make_forum(df, 2022)

0

In [85]:
df1 = pd.read_excel('PHAS Fall 2022 admission.xlsx')

In [87]:
df1.columns

Index(['Acad Group Code', 'Primary Acad Plan Code', 'Acad Prog Code',
       'Degree Plan Code', 'Academic Plan 1 Code', 'Academic Plan 2 Code',
       'Academic Plan 3 Code', 'Person First Name', 'Person Last Name',
       'Student ID', 'Mailing Address', 'Person Phone', 'Person Email Addr',
       'Gender', 'Person Disability Ind', 'Person Aboriginal Ind',
       'Birth Date', 'Country', 'Residency', 'Application Status Desc',
       'Ref 1 First Name', 'Ref 1 Last Name', 'Ref 1 Phone',
       'Ref 1 Organization Name', 'Ref 1 Title', 'Ref 2 First Name',
       'Ref 2 Last Name', 'Ref 2 Phone', 'Ref 2 Organization Name',
       'Ref 2 Title', 'Ref 3 First Name', 'Ref 3 Last Name', 'Ref 3 Phone',
       'Ref 3 Organization Name', 'Ref 3 Title',
       'Are you applying to a combined program? (Example: MBA/JD) If yes, specify which program. ',
       ' Are you looking for advanced credit? If yes, please list the courses you wish to receive credit for.  ',
       'Faculty Question 3', '

In [17]:
html_fields = {
    'personal_info': {
        'first_name': 'Person First Name',
        'last_name': 'Person Last Name',
        'student_number': 'Student ID'
    },
    'program_info': {
        'academic_program': 'Acad Prog Code',
        'academic_plan': 'Academic Plan 1 Code',
        'combined_program': 'Are you applying to a combined program? (Example: MBA/JD) If yes, specify which program. ',
        'advanced_credit': ' Are you looking for advanced credit? If yes, please list the courses you wish to receive credit for.  '
    },
    'residency_info': {
        'country_par': 'Country',
        'residency_par': 'Residency'
    },
    'academic_info': {
        'research_areas': 'research areas',
        'statement_par': 'Statement',
        'gpa_etc': 'GPA etc.',
        'awards_par': 'Awards',
        'financial_support': 'Financial support',
        'job_experience': 'Jobs',
        'publications_par': 'Publications',
        'other_par': 'Other'
    }
}

In [18]:
import json

with open('templates/html_fields.json', 'w') as file:
    json.dump(html_fields, file)

In [19]:
with open('templates/html_fields.json', 'r') as file:
    d1 = json.load(file)

In [20]:
d1

{'personal_info': {'first_name': 'Person First Name',
  'last_name': 'Person Last Name',
  'student_number': 'Student ID'},
 'program_info': {'academic_program': 'Acad Prog Code',
  'academic_plan': 'Academic Plan 1 Code',
  'combined_program': 'Are you applying to a combined program? (Example: MBA/JD) If yes, specify which program. ',
  'advanced_credit': ' Are you looking for advanced credit? If yes, please list the courses you wish to receive credit for.  '},
 'residency_info': {'country_par': 'Country', 'residency_par': 'Residency'},
 'academic_info': {'research_areas': 'research areas',
  'statement_par': 'Statement',
  'gpa_etc': 'GPA etc.',
  'awards_par': 'Awards',
  'financial_support': 'Financial support',
  'job_experience': 'Jobs',
  'publications_par': 'Publications',
  'other_par': 'Other'}}